In [141]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/Heysoos/Documents/Pycharm Projects/Dissertation/02_critical_ising')

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

from tqdm import tqdm

from models.isingCA_global_xi import isingCA

import seaborn as sns
from tqdm.auto import tqdm
plt.style.use('default') # if it's using the wrong style for some reason

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
ft = 10
import matplotlib as mpl
mpl.style.use('default')

# graphical properties
plt.rcParams["axes.edgecolor"] = "k"
plt.rcParams["axes.facecolor"] = "w"
plt.rcParams["axes.linewidth"] = "0.8"
plt.rcParams.update({'font.size': ft})
plt.rcParams['savefig.dpi'] = 300

plt.rcParams['pdf.fonttype'] = 42 # prepare as vector graphic
plt.rcParams['ps.fonttype'] = 42

plt.rcParams["font.family"] = "Helvetica"
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command

C:\Users\Heysoos\AppData\Local\Temp\ipykernel_29080\3538168403.py:17: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command


In [143]:
# resolution of grid
RESX=128
RESY=128
RES = (RESX, RESY)

CHANNELS=1 # number of channels in grid
RADIUS=1
BETA=1/(RADIUS * RADIUS * CHANNELS)
PD=True
BS=256

ca = isingCA(RES, CHANNELS=CHANNELS, BETA=BETA, RADIUS=RADIUS).cuda()

In [144]:
def get_ising_stats_batch(ca, temp_batch, timesteps, eq_steps, save_every=5, PD=True):
    '''
    Get the ising statistics for a batch of temperatures
    :param ca: Ising CA model
    :param temp_batch: the temperatures to get statistics for
    :param timesteps: # of timesteps to get statistics for
    :param eq_steps: # of equilibration steps
    :param save_every: how often to save statistics
    :param PD: periodic boundary conditions (true/false)
    :return: statistics for the batch of temperatures (Energy (E), Magnetization (M), Specific Heat (C), Susceptibility (X))
    '''
    
     # Initialize states for the current batch
    state, obvs = ca.initGrid(BS=len(temp_batch))

    # Modify states based on temperature
    state[:, -1] = torch.ones_like(state[:, -1]) * (1. / torch.tensor(temp_batch, dtype=state.dtype, device=state.device)).view(-1, 1, 1)
    
    # initialize spins to be annealed with the relation: m = (3(T - Tc)/Tc) ** 1/2 for T < Tc
    for i_temp, temp in enumerate(temp_batch):
        if temp < Tc:
            thresh = (3*(temp - Tc)/Tc) ** 1/2 
            state[i_temp, 0] = (torch.rand_like(state[i_temp, 0]) < thresh) * 2. - 1.
    
    # time stretching factor for larger networks to let them equilibrate for good statistics
    time_scale = np.sqrt(np.product(state.shape[-2])/(8 * 8))
    
    # equilibrate for a bit
    for t in range(int(time_scale * eq_steps)):
        state, _ = ca.forward(state, pd=PD)

    all_obvs_batch = []
    for t in range(int(time_scale * timesteps)):
        state, obvs = ca.forward(state, pd=PD)

        if t % save_every == 0:
            all_obvs_batch.append(obvs.cpu().numpy())

    all_obvs_batch = np.stack(all_obvs_batch)
    
    e_t = all_obvs_batch[:, 0, :]
    e2_t = all_obvs_batch[:, 1, :]
    m_t = all_obvs_batch[:, 2, :]
    m2_t = all_obvs_batch[:, 3, :]

    E_t = np.mean(e_t, axis=0)
    M_t = np.mean(m_t, axis=0)
    C_t = (np.mean(e2_t, axis=0) - np.mean(e_t, axis=0) ** 2) * (1./temp_batch) ** 2
    X_t = (np.mean(m2_t, axis=0) - np.mean(m_t, axis=0) ** 2) * (1./temp_batch)

    obvs = np.stack([E_t, M_t, C_t, X_t])
    return obvs

def get_ising_stats_temp(RES, temps, timesteps, eq_steps, num_runs=5, save_every=5, PD=True):
    '''
    Get the ising statistics for a range of temperatures (uses the batched version of get_ising_stats_batch) to
    split the temperatures into batches of size BS
    :param ca: Ising CA model
    :param temps: the temperatures to get statistics for
    :param timesteps: # of timesteps to get statistics for
    :param eq_steps: # of equilibration steps
    :param num_runs: # of runs to average over
    :param save_every: how often to save statistics
    :param PD: periodic boundary conditions (true/false)
    :return: statistics for the range of temperatures (Energy (E), Magnetization (M), Specific Heat (C), Susceptibility (X)
    '''
    all_obvs = []
    # Iterate over temperature batches
    for i in range(0, len(temps), BS):
        temp_batch = temps[i:i + BS]
        
        ca = isingCA(RES, CHANNELS=CHANNELS, BETA=BETA, RADIUS=RADIUS).cuda()
        
        all_obvs_run = []
        for j in tqdm(range(num_runs)):
            all_obvs_run.append(get_ising_stats_batch(ca, temp_batch, timesteps, eq_steps, save_every=save_every, PD=PD))
        all_obvs.append(np.stack(all_obvs_run))
    all_obvs = np.concatenate(all_obvs, axis=-1)
    
    return all_obvs
    

In [145]:
temps = np.geomspace(1.8, 3.5, BS)
eq_steps = 2 * 10 ** 3
timesteps = 10 ** 5
num_runs = 10

# stats to save
save_every = 10
all_obvs = []

Tc = 2.26924

all_obvs = get_ising_stats_temp((RESX, RESY), temps, timesteps, eq_steps, num_runs=num_runs, save_every=save_every, PD=PD)

 30%|███       | 3/10 [59:00<2:17:41, 1180.27s/it]


KeyboardInterrupt: 

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(4.5, 4.5), dpi=120)

N = (RESX * RESY)

# [num_runs, 4, temperature]
E = np.mean(all_obvs[:, 0, :], axis=0)
M = np.mean(all_obvs[:, 1, :], axis=0)
C = np.mean(all_obvs[:, 2, :], axis=0) / N
X = N * np.mean(all_obvs[:, 3, :], axis=0)
# C = all_obvs[:, 2, :] / N
# X = N * all_obvs[:, 3, :]

marker = '-'
ms = 1.
lw=3
alpha = 0.8

axes[0, 0].plot(temps, E/N, marker, ms=ms, c='k', alpha=alpha, lw=lw)
axes[1, 0].plot(temps, np.abs(M), marker, ms=ms, c='k', alpha=alpha, lw=lw)
axes[0, 1].plot(temps, C, marker, ms=ms, c='k', alpha=alpha, lw=lw)
axes[1, 1].plot(temps, X, marker, ms=ms, c='k', alpha=alpha, lw=lw)


axes[0, 0].set_title(r'$E/N$')
axes[1, 0].set_title(r'$M$')
axes[0, 1].set_title(r'$C$')
axes[1, 1].set_title(r'$\chi$')

data = [E, C, np.abs(M), X]
lw = 2
for i, ax in enumerate(axes.flatten()):
    y_min, y_max = ax.get_ylim()
    ax.vlines(Tc, y_min, y_max, 'grey', '--', lw=lw, alpha=0.8, label=rf'$T_c$')
    ax.set_xticks(list(ax.get_xticks())[1:-1] + [Tc], minor=True, labels=list(ax.get_xticklabels())[1:-1] + [r'$T_c$'])
axes[0, 0].legend(loc='lower right', frameon=False)

axes[1, 1].set_yscale('log')

plt.tight_layout()
plt.savefig('../figures/ising_stats.pdf', bbox_inches='tight')

### Again for different size networks

In [ ]:
# sizes = [2 ** i for i in range(4, 11)]
sizes = [8, 16, 32, 64, 128]

# if using actual temperature
temps = np.geomspace(1.8, 3.5, BS)

# if using reduced temperature
# exp_nu = 1
# reduced_temps = np.linspace(-2, 2, 16*6)

# simulation statistics
eq_steps = 2 * 10 ** 3
timesteps = 10 ** 5
num_runs = 10

# stats to save
save_every = 10
all_obvs_multi = []

for size in sizes:
    print(f'Starting size = {size}')
    RES = (size, size)
    all_obvs = get_ising_stats_temp((size, size), temps, timesteps, eq_steps, num_runs=num_runs, save_every=save_every, PD=PD)
    all_obvs_multi.append(all_obvs)
    
all_obvs_multi = np.stack(all_obvs_multi)

Starting size = 8


100%|██████████| 10/10 [13:42<00:00, 82.23s/it]


Starting size = 16


100%|██████████| 10/10 [19:37<00:00, 117.72s/it]


Starting size = 32


100%|██████████| 10/10 [35:00<00:00, 210.00s/it]


Starting size = 64


 60%|██████    | 6/10 [3:56:48<2:21:27, 2121.87s/it]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(4.5, 4.5), dpi=300)

c_skip = 2
colors = sns.color_palette("flare", len(sizes) + c_skip)[c_skip:][::-1]


for i_s, size in enumerate(sizes):  
    N = (size * size)
    
    E = np.mean(all_obvs_multi[i_s][:, 0, :], axis=0)
    M = np.mean(all_obvs_multi[i_s][:, 1, :], axis=0)
    
    C = np.mean(all_obvs_multi[i_s][:, 2, :], axis=0) / N
    X = N * np.mean(all_obvs_multi[i_s][:, 3, :], axis=0)

    marker = '.-'
    ms=1.
    c = colors[i_s]
    alpha = 0.8
    axes[0, 0].plot(temps, E/N, marker, ms=ms, c=c, alpha=alpha)
    axes[1, 0].plot(temps, np.abs(M), marker, ms=ms, c=c, alpha=alpha)
    axes[0, 1].plot(temps, C, marker, ms=ms, label=rf'${size}$', c=c, alpha=alpha)
    axes[1, 1].plot(temps, X, marker, ms=ms, c=c, alpha=alpha)


    axes[0, 0].set_title(r'$E/N$')
    axes[1, 0].set_title(r'$M$')
    axes[0, 1].set_title(r'$C/N$')
    axes[1, 1].set_title(r'$\chi$')
    

for i, ax in enumerate(axes.flatten()):
    y_min, y_max = ax.get_ylim()
    ax.vlines(Tc, y_min, y_max, 'grey', '--', alpha=0.8, label=r'$T_c$' if i == 0 else None)
    ax.set_xticks(list(ax.get_xticks())[1:-1] + [Tc], minor=True, labels=list(ax.get_xticklabels())[1:-1] + [r'$T_c$'], fontsize=12, color='grey')
# axes[0, 0].legend(loc='lower right', frameon=False)
axes[0, 1].legend(frameon=False, borderpad=0.1, labelspacing=0.05, loc='upper right', ncol=1)

axes[1, 1].set_yscale('log')
# axes[0, 1].set_yscale('log')
axes[1, 1].set_xlabel(r'$T$')
axes[1, 0].set_xlabel(r'$T$')

plt.tight_layout()
plt.savefig('../figures/ising_stats_multiscale.pdf', bbox_inches='tight')

## Collapse the curves

Rescaled temperature:
$$ t = \frac{(T - T_c)}{T_C}$$
$$ \tilde{t} = t L^{1/\nu} $$

Rescaled Specific Heat:
$$ \tilde{C}(t, L) = L^{-\alpha/\nu} C(t, L) $$

Rescaled Susceptibility:
$$ \tilde{\chi}(t, L) = L^{-\gamma/\nu} \chi(t, L) $$

where $T_c = 2.2691$, $\alpha = 0$, $\gamma = 7/4$, and $\nu = 1$ is known for the 2D Ising model.

In [ ]:
exp_alpha = 0
exp_nu = 1
exp_gamma = 7/4

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(6, 3), dpi=120)

c_skip = 2
# colors = sns.color_palette("flare", len(sizes) + c_skip)[c_skip:][::-1]
for i_s, size in enumerate(sizes):
    
    # define the rescaling factors
    rescaled_temps = ((temps - Tc) / Tc) * (size ** (1./exp_nu))

    C_L = size ** (-exp_alpha / exp_nu)
    X_L = size ** (-exp_gamma / exp_nu)
    
    N = (size * size)
    C = C_L * np.mean(all_obvs_multi[i_s][:, 2, :], axis=0) / N
    X = X_L * np.mean(N * all_obvs_multi[i_s][:, 3, :], axis=0)
    
    # C = [C_L * size * size * obv[2] for obv in all_obvs_multi[i_s]]
    # X = [X_L * size * size * obv[3] for obv in all_obvs_multi[i_s]]

    marker = '.'
    ms=1.
    lw=2
    axes[0].plot(rescaled_temps, C, ms=ms, label=rf'${size}$', c=colors[i_s], lw=lw)
    axes[1].plot(rescaled_temps, X, ms=ms, c=colors[i_s], lw=lw)
    # axes[0].vlines(Tc, np.min(C), np.max(C)*10, 'r', '--', lw=lw, alpha=0.8, label=rf'$T_c$')
    # axes[1].vlines(Tc, np.min(C), np.max(C)*10, 'r', '--', lw=lw, alpha=0.8, label=rf'$T_c$')

    
axes[0].set_xlabel(r'$tL^{1 / \nu}$', fontsize=12)
axes[1].set_xlabel(r'$tL^{1 / \nu}$', fontsize=12)

axes[0].set_ylabel(r'$C L^{-\alpha / \nu}$', fontsize=12)
axes[1].set_ylabel(r'$\chi L^{-\gamma / \nu}$', fontsize=12)
    
    # axes[0].set_yscale('log')
    # axes[1].set_yscale('log')

for i, ax in enumerate(axes.flatten()):
    y_min, y_max = ax.get_ylim()
    ax.vlines(0, y_min, y_max, 'grey', '--', alpha=0.8)
axes[0].legend(frameon=False, borderpad=0.1, labelspacing=0.05)

for ax in axes:
    ax.set_xlim([-10, 10])
plt.tight_layout()
plt.savefig('../figures/collapsed_ising_stats.pdf', bbox_inches='tight')

Doesn't seem to curve-collapse very well :(